# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
import pandas_profiling as pdp

# magic word for producing visualizations in notebook
%matplotlib inline

# Setup some nice defaults for dataframe display
pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
gen_pop = pd.read_csv('data/Udacity_AZDIAS_Subset.csv', sep=';')
gen_pop.head()

In [ ]:
# Load in the feature summary file.
feat_info = pd.read_csv('data/AZDIAS_Feature_Summary.csv', sep=';')
feat_info

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).

gen_pop.info(memory_usage='deep')

**That's weird. This says that I have 891,221 rows but the data dictionary and Udacity project page indicate that there should be 10 fewer entries than that, 891,211.** A typo in the data dictionary page perhaps? Or maybe the data have been updated? 

Or perhaps something more nefarious?

In [ ]:
# Do some quick automated EDA
# This will help especially in indicating the original amount of naturally missing data
# prior to mapping the missing/unknown value codes to np.nan

pdp.ProfileReport(gen_pop)


## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# Look at naturally missing data first

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace=True)
missing

In [ ]:
none_missing_count = len(missing[missing['total missing'] > 0])
print(
    f"{none_missing_count} columns have a nonzero number of naturally missing values")


**Oof, looks like the consumer behavior type (`KK_KUNDENTYP`) is pretty sorely lacking values.** Hard to say if it will be the dominant "missing value" feature once we fill in the coded missing values with `np.nan`, but given that it's more than 60% missing I'll probably need to drop it as a feature. In my experience, features with that many missing values are rarely useful for modeling.

In [ ]:
# How many rows have at least one missing value?
rows_oneMissing = len(gen_pop) - len(gen_pop.dropna(how='any'))
rows_allMissing = len(gen_pop) - len(gen_pop.dropna(how='all'))

print(f"There are {rows_oneMissing} samples with at least one feature missing a value \
\nand {rows_allMissing} samples missing values for all features")


In [ ]:
# Convert the strings for the missing values from the feature summary
# To be proper lists of values to use for filling in NaNs

# First, remove brackets
# Then, split on comma separator
feat_info.loc[:, 'missing_or_unknown'] = \
    feat_info.loc[:, 'missing_or_unknown'].str[1:-1].str.split(',')

feat_info


In [ ]:
# TODO: create parser function for making the lists as I did so above

In [ ]:
# What columns are object (probably mixed) type?
gen_pop.columns[gen_pop.dtypes == 'object']

In [ ]:
# What unique data types exist within the object-type columns?
col_types = {}

for col in gen_pop.columns[gen_pop.dtypes == 'object']:
    types = set()
    col_types[col] = set([type(e) for e in gen_pop[col]])
    
col_types

**Good to know! Looks like all object-type columns are of mixed float and str type**. This will help us do intelligent typecasting as we parse through the `feat_info` missing codes below.

A few other notes (ignoring codes used for missing):

1. `OST_WEST_KZ`
    1. Can be "O" or "W"
2. `CAMEO_DEUG_2015`
    1. All int
3. `CAMEO_DEU_2015`
    1. Alphanumeric (of form INT-LETTER)
4. `CAMEO_INTL_2015`
    1. All int

In [ ]:
def fill_missing(df, missing_codes_mapping, inplace=False):
    '''
    Parses dataframe of missing values and their mapping to individual feature names
    and then fills any of those values found in a dataframe's matching feature columns
    with np.nan.

    Inputs
    ------
    df: pandas DataFrame. Table with features that match the ones for which we have
        missing mappings. Each sample is a person.

    missing_codes_mapping: pandas DataFrame. Contains columns 'attribute' and 
        'missing_or_unknown' that map codes used for missing/unknown values to 
        features/attributes. 'missing_or_unknown' is expected to have elements
        that are lists of str (usually ints, but sometimes chars or empty lists).

    Returns
    -------
    df with NaN values filled in according to missing_codes_mapping
    '''

    # Use deep copy if inplace = False, otherwise use shallow copy
    data = df.copy(deep=not inplace)
    missing_codes = missing_codes_mapping.copy(deep=not inplace)

    def parse_missing_codes(code_list):
        '''
        Goes through a list of str and converts the elements of the list according to the needs 
        of the dtypes in our demographic data such that the results can be used for 
        filling in NaN values.

        Inputs
        ------
        code_list: list of str. List is expected to contain the chars, floats, or ints 
            that are codes indicating a missing or unknown value.

        Returns
        -------
        list or np.nan. Each element of the list returned is typecast according to 
            the expected needs of the NaN-filling it will be doing. Empty lists
            (or lists with only an empty string in them) are returned as np.nan.
        '''

        # Make sure list isn't just empty string
        if '' not in code_list:
            # Check if list can be converted to int without issues - if so, do it
            try:
                return [int(e) for e in code_list]

            # Not all are cast-able to int
            except ValueError:
                return [float(e) if 'X' not in e else e for e in code_list]

        else:
            return np.nan

    # Typecast missing value codes appropriately
    missing_codes.loc[:, 'missing_or_unknown'] = \
        missing_codes.loc[:, 'missing_or_unknown'].apply(parse_missing_codes)

    # Create series that maps feature names (index) to missing codes (data)
    code_map = pd.Series(data=missing_codes['missing_or_unknown'].values,
                         index=missing_codes['attribute'].values)

    # When passing a Series into to_replace, index is key and data is value (like a dict)
    data.replace(to_replace=code_map,
                 value=np.nan,
                 inplace=True)

    return data


In [ ]:
# Replace codes we know to mean "missing" or "unknown" with NaN
gen_pop = fill_missing(gen_pop, feat_info)
gen_pop.head()

In [ ]:
# Save a copy of gen_pop now that we've cleaned up its missing data so we can avoid doing work again
gen_pop.to_csv('data/gen_pop.csv')

In [ ]:
# TODO: make sure when importing customer data to use same dtypes as gen_pop

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Do some quick automated EDA
# This will help especially in indicating the original amount of naturally missing data
# prior to mapping the missing/unknown value codes to np.nan

pdp.ProfileReport(gen_pop)


In [ ]:
# Automated EDA again now that we've encoded our missing values as np.nan properly

pdp.ProfileReport(gen_pop)

In [ ]:
# Look at naturally missing data first

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace=True)
missing

In [ ]:
# What do the missing data counts look like now, by column?

missing_filled = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing_filled['percent missing'] = round(missing_filled['total missing'] / len(gen_pop),2)
missing_filled.sort_values('total missing', ascending = False, inplace=True)
missing_filled

In [ ]:
(missing['percent missing'] == 0.0).sum()

**Well that's a non-trivial change!** The first profile I generated prior to filling in missing values had 6.4% of all values missing, now we're at 10.6%! Additionally:

1. Before NaN-filling, we only had one feature with more than 15% of values missing (`KK_KUNDENTYP`) and 32 features with no missing values at all. 
    1. After filling in the NaN codes with `np.nan`, we end up with 9 having more than 15% of values missing and 25 with no missing values.
1. `AGER_TYPE` has **77% of its values missing.** It wasn't even flagged as special prior to the NaN-filling step.
2. `ALTER_HH` went from 8.2% missing to **34.8% missing!**
3. Other examples exist (6 more I haven't mentioned that are above 15% missing)

In [ ]:
none_missing_count = len(missing[missing['total missing'] > 0])
print(
    f"{none_missing_count} columns have a nonzero number of missing values now")


In [ ]:
# Visualize the missing count across columns

sns.distplot(missing['total missing'], bins = 25)
#sns.barplot(x=missing.index, y='percent missing', data=missing)

**It seems pretty clear here that any features we should consider as "outliers" are the ones that have more than 20% missing values. Let's remove those.**

In [ ]:
missing[missing['total missing'] > 2E5]

In [ ]:
# Remove the outlier columns from the dataset

outlier_cols = list(missing[missing['percent missing'] > 0.2].index.values)
gen_pop.drop(columns=outlier_cols, inplace=True)

In [ ]:
# Re-assess after dropping outlier features

missing_noOutliers = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing_noOutliers['percent missing'] = round(missing_noOutliers['total missing'] / len(gen_pop),2)
missing_noOutliers.sort_values('total missing', ascending = False, inplace=True)
missing_noOutliers

In [ ]:
# What does missing count distribution look like now?

sns.distplot(missing_noOutliers['total missing'],
            bins = 25)

In [ ]:
missing_noSort = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})

missing_noSort[missing_noSort['total missing'] < 50000]
#missing_noSort[missing_noSort['total missing'] > 50000]

In [ ]:
len(missing_noSort[missing_noSort['total missing'] < 50000])

In [ ]:
# What % of all features is represented at different information levels?

feat_info['information_level'].value_counts()/len(feat_info)

In [ ]:
# Parse how many features are named something common to the 
# low-missing-values sample relative to the total feature space

feat_snippet = 'LP_'

low_missing_num = missing_noSort[missing_noSort['total missing'] < 50000].index\
.str.contains(feat_snippet).sum()
total = feat_info['attribute'].str.contains(feat_snippet).sum()

print(f"There are {low_missing_num} features in the low-missing-values \
feature space with {feat_snippet} in their name, out of a total possible {total} \
features with the same thing in their name.")

In [ ]:
# First things first: if I'm going to make sense out of these, I need more helpful names

new_names = {
    'AGER_TYP': 'Elderly Type',
    'ALTERSKATEGORIE_GROB': 'Age Bin',
    'ANREDE_KZ': 'Gender',
    'CJT_GESAMTTYP': 'Consumption Channel Type',
    'FINANZ_MINIMALIST': 'MoneyType__Minimalist',
    'FINANZ_SPARER': 'MoneyType__Saver',
    'FINANZ_VORSORGER': 'MoneyType__Preparer',
    'FINANZ_ANLEGER': 'MoneyType__Investor',
    'FINANZ_UNAUFFAELLIGER': 'MoneyType__Inconspicuous',
    'FINANZ_HAUSBAUER': 'MoneyType__Homeowner',
    'FINANZTYP': 'MoneyType__Primary',
    'GEBURTSJAHR': 'Birth Year',
    'GFK_URLAUBERTYP': 'Vacation Habits',
    'GREEN_AVANTGARDE': 'Young Environmentalist',
    'HEALTH_TYP': 'Health Type',
    'LP_LEBENSPHASE_FEIN': 'Life Stage - HighRes',
    'LP_LEBENSPHASE_GROB': 'Life Stage - LowRes',
    'LP_FAMILIE_FEIN': 'Family Type - HighRes',
    'LP_FAMILIE_GROB': 'Family Type - LowRes',
    'LP_STATUS_FEIN': 'Socioeconomic Status - HighRes',
    'LP_STATUS_GROB': 'Socioeconomic Status - LowRes',
    'NATIONALITAET_KZ': 'Nationality Based on Name',
    'PRAEGENDE_JUGENDJAHRE': 'Generation Designation',
    'RETOURTYP_BK_S': 'Customer Type',
    'SEMIO_SOZ': 'PersonalityType__Social',
    'SEMIO_FAM': 'PersonalityType__Family',
    'SEMIO_REL': 'PersonalityType__Religious',
    'SEMIO_MAT': 'PersonalityType__Materialist',
    'SEMIO_VERT': 'PersonalityType__Dreamer',
    'SEMIO_LUST': 'PersonalityType__Sensual',
    'SEMIO_ERL': 'PersonalityType__Event',
    'SEMIO_KULT': 'PersonalityType__Cultured',
    'SEMIO_RAT': 'PersonalityType__Rational',
    'SEMIO_KRIT': 'PersonalityType__Critical',
    'SEMIO_DOM': 'PersonalityType__Dominant',
    'SEMIO_KAEM': 'PersonalityType__Combative',
    'SEMIO_PFLICHT': 'PersonalityType__Dutiful',
    'SEMIO_TRADV': 'PersonalityType__Traditional',
    'SHOPPER_TYP': 'Shopper Type',
    'SOHO_KZ': 'Small or Home Office Owner',
    'TITEL_KZ': 'Academic Title',
    'VERS_TYP': 'Insurance Type',
    'ZABEOTYP': 'Energy Consumption Type',
    'ALTER_HH': 'HH: Head of HH Birthdate Bin (5-Year)',
    'ANZ_PERSONEN': 'HH: Number of Adults in HH',
    'ANZ_TITEL': 'HH: Number of Academic Title Holders in HH',
    'HH_EINKOMMEN_SCORE': 'HH: Net Income Bins',
    'KK_KUNDENTYP': 'HH: Customer Type',
    'W_KEIT_KIND_HH': 'HH: Probability of Children in Residence',
    'WOHNDAUER_2008': 'HH: Length of Residency (bins)',
    'NZ_HAUSHALTE_AKTIV': 'Bldg: Number of HHs',
    'ANZ_HH_TITEL': 'Bldg: Number of Academic Title Holders',
    'GEBAEUDETYP': 'Bldg: Building Type',
    'KONSUMNAEHE': 'Bldg: Distance to Point of Sale Category',
    'MIN_GEBAEUDEJAHR': "Bldg: Year of Building's Initial Database Entry",
    'OST_WEST_KZ': 'Bldg: Location Relative to E or W Germany',
    'WOHNLAGE': 'Bldg: Neighborhood Quality',
    'CAMEO_DEUG_2015': 'RR4: Life Stage Type - LowRes',
    'CAMEO_DEU_2015': 'RR4: Life Stage Type - HighRes',
    'CAMEO_INTL_2015': "RR4: Life Stage Type - Int'l Code Mapping",
    'KBA05_ANTG1': 'RR3: Bins of 1-2 Family Homes',
    'KBA05_ANTG2': 'RR3: Bins of 3-5 Family Homes',
    'KBA05_ANTG3': 'RR3: Bins of 6-10 Family Homes',
    'KBA05_ANTG4': 'RR3: Bins of 10+ Family Homes',
    'KBA05_BAUMAX': 'RR3: Most Common Bldg Type',
    'KBA05_GBZ': 'RR3: Bin Counts of Bldgs',
    'BALLRAUM': 'PostCode: Distance to Nearest Urban Center (bins)',
    'EWDICHTE': 'PostCode: Density of HHs per km^2 (bins)',
    'INNENSTADT': 'PostCode: Distance to City Center (bins)',
    'GEBAEUDETYP_RASTER': 'RR1: Residential-Commercial Activity Ratio (categories)',
    'KKK': 'RR1: Purchasing Power (bins)',
    'MOBI_REGIO': 'RR1: Movement Patterns',
    'ONLINE_AFFINITAET': 'RR1: Online Affinity',
    'REGIOTYP': 'RR1: Neighborhood Type',
    'KBA13_ANZAHL_PKW': 'PLZ8: Number of Cars',
    'PLZ8_ANTG1': 'PLZ8: Bins of 1-2 Family Homes',
    'PLZ8_ANTG2': 'PLZ8: Bins of 3-5 Family Homes',
    'PLZ8_ANTG3': 'PLZ8: Bins of 6-10 Family Homes',
    'PLZ8_ANTG4': 'PLZ8: Bins of 10+ Family Homes',
    'PLZ8_BAUMAX': 'PLZ8: Most Common Bldg Type',
    'PLZ8_HHZ': 'PLZ8: Bin Counts of HHs',
    'LZ8_GBZ': 'PLZ8: Bin Counts of Bldgs',
    'ARBEIT': 'Community: Share of Unemployment',
    'ORTSGR_KLS9': 'Community: Size (bins)',
    'RELAT_AB': 'Community: Share of Unemployment Relative to Parent County'
}

gen_pop.rename(columns=new_names, inplace=True)


#### Discussion 1.1.2: Assess Missing Data in Each Column

Here are my notes on what just went down in this section:

1. Before matching codes for unknown/missing values and filling in those values with `np.nan`, we had 6.4% of all values reported as missing. Afterwards, 10.6% of all values were missing.
2. I removed features with more than 20% missing values as they appeared to be outliers in terms of the amount of missing values relative to the overall feature set
    * Specifically, the features removed were `TITEL_KZ`, `AGER_TYP`, `KK_KUNDENTYP`, `KBA05_BAUMAX`, `GEBURTSJAHR`, and `ALTER_HH`
3. In the data that are remaining, I see a clear breakpoint between features with 0% - 2% missing values and those with 8% - 18% missing values.
    * Those in the group with 0% - 2%:
        * All but two (`ONLINE_AFFINITAET` and `HH_EINKOMMEN_SCORE`) are person-level measures. Given that there are 32 features in this subset and ~50% of all features are person-level, this suggests that person-level features are overrepresented in the subset (if the information levels were distributed the same in this sample as they are in the total population, we'd expect only 16-17 features to be person-level). 
            * Also, this means that 70% of the total 43 possible person-level features in the full feature space are represented in this sample.
        * All of the `SEMIO` features are included (personality dimension ratings)
        * All of the `FINANZ` features are included (financial behavorial dimensions)
        * 
    * Those in the group with 8% - 18% missing seem to all be 

In [ ]:
(feat_info['information_level'] == 'person').sum()

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# Remind me, how many rows had missing data prior to NaN-filling?

print(f"There are {rows_oneMissing} samples with at least one feature missing a value \
\nand {rows_allMissing} samples missing values for all features")

In [ ]:
# What do the missing data counts look like now, by row?

rows_oneMissing_filled = len(gen_pop) - len(gen_pop.dropna(how='any'))
rows_allMissing_filled = len(gen_pop) - len(gen_pop.dropna(how='all'))

print(f"There are {rows_oneMissing_filled} samples with at least one feature missing a value \
\nand {rows_allMissing_filled} samples missing values for all features")

**That's quite the change! Almost every sample (all but 143 of them) has at least one missing value for a feature!**

In [ ]:
# How much data is missing in each row of the dataset?



In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.



In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.



#### Discussion 1.1.3: Assess Missing Data in Each Row

(Double-click this cell and replace this text with your own text, reporting your observations regarding missing data in rows. Are the data with lots of missing values are qualitatively different from data with few or no missing values?)

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?



#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?



In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.



#### Discussion 1.2.1: Re-Encode Categorical Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding categorical features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.



In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.



#### Discussion 1.2.2: Engineer Mixed-Type Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding mixed-value features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# If there are other re-engineering tasks you need to perform, make sure you
# take care of them here. (Dealing with missing data will come in step 2.1.)



In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.



### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    
    # Put in code here to execute all main cleaning steps:
    # convert missing value codes into NaNs, ...
    
    
    # remove selected columns and rows, ...

    
    # select, re-encode, and engineer column values.

    
    # Return the cleaned dataframe.
    
    

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.



In [ ]:
# Apply feature scaling to the general population demographics data.



### Discussion 2.1: Apply Feature Scaling

(Double-click this cell and replace this text with your own text, reporting your decisions regarding feature scaling.)

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.



In [ ]:
# Investigate the variance accounted for by each principal component.



In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.



### Discussion 2.2: Perform Dimensionality Reduction

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding dimensionality reduction. How many principal components / transformed features are you retaining for the next step of the analysis?)

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.



In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.



In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.



### Discussion 2.3: Interpret Principal Components

(Double-click this cell and replace this text with your own text, reporting your observations from detailed investigation of the first few principal components generated. Can we interpret positive and negative values from them in a meaningful way?)

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Over a number of different cluster counts...


    # run k-means clustering on the data and...
    
    
    # compute the average within-cluster distances.
    
    

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.



In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.



### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = 

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.



### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.



In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.